In [ ]:
# dataframe 을 for 루프 돌면서 multiMovieYn, multiMovieYn 값이 NaN 이면
#dataframe 전체 에서 같은 movieCd row 에서 해당 값을 가져오자

In [20]:
import pandas as pd
PATH = "~/data/movies/dailyboxoffice/dt=20240101"
df = pd.read_parquet(PATH)
len(df)

df.groupby(['multiMovieYn', 'repNationCd'], dropna=False, observed=False) \
.size() \
.reset_index(name='count')

,multiMovieYn,repNationCd,count
0,N,F,0
1,N,K,0
2,N,NaN,10
3,Y,F,0
4,Y,K,0
5,Y,NaN,10
6,NaN,F,10
7,NaN,K,10
8,NaN,NaN,10


In [21]:
for _, row in df.iterrows():
    if pd.isna(row['multiMovieYn']):
        print(f"{row['movieCd']}")

20203702
20212866
20236146
20235735
20235596
20234673
20235290
20234114
20236045
20235980
20236146
20235735
20235596
20234114
20236045
20235980
20236080
20233299
20235923
20235693
20203702
20212866
20234673
20235290
20190324
20202247
20228560
20235697
20232077
20227525


In [22]:
na_movie_cds = []
for _, row in df.iterrows():
    if pd.isna(row['multiMovieYn']):
        na_movie_cds.append(row['movieCd'])
print(na_movie_cds)

['20203702', '20212866', '20236146', '20235735', '20235596', '20234673', '20235290', '20234114', '20236045', '20235980', '20236146', '20235735', '20235596', '20234114', '20236045', '20235980', '20236080', '20233299', '20235923', '20235693', '20203702', '20212866', '20234673', '20235290', '20190324', '20202247', '20228560', '20235697', '20232077', '20227525']


In [27]:
import pandas as pd

PATH = "~/data/movies/dailyboxoffice/dt=20240101"
df = pd.read_parquet(PATH)

# 첫 번째 그룹화 (multiMovieYn, repNationCd 별 count)
grouped_counts = df.groupby(['multiMovieYn', 'repNationCd'], dropna=False, observed=False) \
    .size() \
    .reset_index(name='count')

print("multiMovieYn, repNationCd 별 count:")
print(grouped_counts.to_markdown(index=False))
print("-" * 30)

# 'multiMovieYn'이 NaN인 movieCd 추출 (중복 제거)
na_movie_cds = df[pd.isna(df['multiMovieYn'])]['movieCd'].unique().tolist()
print("'multiMovieYn'이 NaN인 고유한 movieCd 목록:", na_movie_cds)
print("-" * 30)

results = []
for movie_cd in na_movie_cds:
    r = df[df['movieCd'] == movie_cd][['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']]
    results.append(r)

consolidated_na_movies_df = pd.DataFrame()
if results:
    combined_df = pd.concat(results, ignore_index=True)

    def consolidate_movie_info(group):
        first_row = group.iloc[0].copy()
        for col in ['multiMovieYn', 'repNationCd']:
            non_na_value = group[pd.notna(group[col])][col].iloc[0] if any(pd.notna(group[col])) else None
            if non_na_value is not None:
                first_row[col] = non_na_value
        return first_row[['movieNm', 'multiMovieYn', 'repNationCd']]

    consolidated_na_movies_df = combined_df.groupby('movieCd').apply(consolidate_movie_info).reset_index()
    print("'multiMovieYn'이 NaN이었던 영화 중 NaN이 아닌 값으로 대체된 정보 (movieCd 기준 중복 제거):")
    print(consolidated_na_movies_df[['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']].to_markdown(index=False))
else:
    print("'multiMovieYn'이 NaN인 movieCd에 해당하는 영화 정보가 없습니다.")

print("-" * 30)
print("전체 영화 정보 (movieCd 기준 NaN 값 대체):")
def consolidate_all_movie_info(group):
    first_row = group.iloc[0].copy()
    for col in ['multiMovieYn', 'repNationCd']:
        non_na_value = group[pd.notna(group[col])][col].iloc[0] if any(pd.notna(group[col])) else None
        if non_na_value is not None:
            first_row[col] = non_na_value
    return first_row[['movieNm', 'multiMovieYn', 'repNationCd']]

all_consolidated_df = df.groupby('movieCd').apply(consolidate_all_movie_info).reset_index()
print(all_consolidated_df[['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']].to_markdown(index=False))

multiMovieYn, repNationCd 별 count:
| multiMovieYn   | repNationCd   |   count |
|:---------------|:--------------|--------:|
| N              | F             |       0 |
| N              | K             |       0 |
| N              | nan           |      10 |
| Y              | F             |       0 |
| Y              | K             |       0 |
| Y              | nan           |      10 |
| nan            | F             |      10 |
| nan            | K             |      10 |
| nan            | nan           |      10 |
------------------------------
'multiMovieYn'이 NaN인 고유한 movieCd 목록: ['20203702', '20212866', '20236146', '20235735', '20235596', '20234673', '20235290', '20234114', '20236045', '20235980', '20236080', '20233299', '20235923', '20235693', '20190324', '20202247', '20228560', '20235697', '20232077', '20227525']
------------------------------
'multiMovieYn'이 NaN이었던 영화 중 NaN이 아닌 값으로 대체된 정보 (movieCd 기준 중복 제거):
|   movieCd | movieNm                                          

/tmp/ipykernel_2567532/2994954350.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  consolidated_na_movies_df = combined_df.groupby('movieCd').apply(consolidate_movie_info).reset_index()
/tmp/ipykernel_2567532/2994954350.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_consolidated_df = df.groupby('movieCd').apply(consolidate_all_movie_info).reset_index()


In [28]:
consolidated_na_movies_df[['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']]

,movieCd,movieNm,multiMovieYn,repNationCd
0,20190324,3일의 휴가,N,K
1,20202247,싱글 인 서울,NaN,K
2,20203702,노량: 죽음의 바다,N,K
3,20212866,서울의 봄,N,K
4,20227525,교토에서 온 편지,NaN,K
5,20228560,너와 나,NaN,K
6,20232077,어른 김장하,NaN,K
7,20233299,크레센도,Y,F
8,20234114,괴물,Y,F
9,20234673,뽀로로 극장판 슈퍼스타 대모험,N,K


In [29]:
all_consolidated_df[['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']]

,movieCd,movieNm,multiMovieYn,repNationCd
0,19860125,쿼바디스,Y,NaN
1,20124722,피아니스트의 전설,Y,NaN
2,20190324,3일의 휴가,N,K
3,20202247,싱글 인 서울,NaN,K
4,20203702,노량: 죽음의 바다,N,K
5,20212866,서울의 봄,N,K
6,20227525,교토에서 온 편지,NaN,K
7,20228560,너와 나,NaN,K
8,20232077,어른 김장하,NaN,K
9,20233299,크레센도,Y,F


In [30]:
import pandas as pd

PATH = "~/data/movies/dailyboxoffice/dt=20240101"
df = pd.read_parquet(PATH)

# 첫 번째 그룹화 (multiMovieYn, repNationCd 별 count)
grouped_counts = df.groupby(['multiMovieYn', 'repNationCd'], dropna=False, observed=False) \
    .size() \
    .reset_index(name='count')

print("multiMovieYn, repNationCd 별 count:")
print(grouped_counts.to_markdown(index=False))
print("-" * 30)


print("-" * 30)
print("전체 영화 정보 (movieCd 기준 NaN 값 대체):")
def consolidate_all_movie_info(group):
    first_row = group.iloc[0].copy()
    for col in ['multiMovieYn', 'repNationCd']:
        non_na_value = group[pd.notna(group[col])][col].iloc[0] if any(pd.notna(group[col])) else None
        if non_na_value is not None:
            first_row[col] = non_na_value
    return first_row[['movieNm', 'multiMovieYn', 'repNationCd']]

all_consolidated_df = df.groupby('movieCd').apply(consolidate_all_movie_info).reset_index()
print(all_consolidated_df[['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']].to_markdown(index=False))

multiMovieYn, repNationCd 별 count:
| multiMovieYn   | repNationCd   |   count |
|:---------------|:--------------|--------:|
| N              | F             |       0 |
| N              | K             |       0 |
| N              | nan           |      10 |
| Y              | F             |       0 |
| Y              | K             |       0 |
| Y              | nan           |      10 |
| nan            | F             |      10 |
| nan            | K             |      10 |
| nan            | nan           |      10 |
------------------------------
------------------------------
전체 영화 정보 (movieCd 기준 NaN 값 대체):
|   movieCd | movieNm                                                       | multiMovieYn   | repNationCd   |
|----------:|:--------------------------------------------------------------|:---------------|:--------------|
|  19860125 | 쿼바디스                                                      | Y              | nan           |
|  20124722 | 피아니스트의 전설                        

/tmp/ipykernel_2567532/1606524095.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_consolidated_df = df.groupby('movieCd').apply(consolidate_all_movie_info).reset_index()


In [31]:
all_consolidated_df[['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']]

,movieCd,movieNm,multiMovieYn,repNationCd
0,19860125,쿼바디스,Y,NaN
1,20124722,피아니스트의 전설,Y,NaN
2,20190324,3일의 휴가,N,K
3,20202247,싱글 인 서울,NaN,K
4,20203702,노량: 죽음의 바다,N,K
5,20212866,서울의 봄,N,K
6,20227525,교토에서 온 편지,NaN,K
7,20228560,너와 나,NaN,K
8,20232077,어른 김장하,NaN,K
9,20233299,크레센도,Y,F


In [48]:
import pandas as pd

PATH = "~/data/movies/dailyboxoffice/dt=20240101"
df = pd.read_parquet(PATH)

# 첫 번째 그룹화 (multiMovieYn, repNationCd 별 count)
grouped_counts = df.groupby(['multiMovieYn', 'repNationCd'], dropna=False, observed=False) \
    .size() \
    .reset_index(name='count')

print("multiMovieYn, repNationCd 별 count:")
#print(grouped_counts.to_markdown(index=False))
print(grouped_counts.to_string(index=False))
print("-" * 30)


print("-" * 30)
print("전체 영화 정보 (movieCd 기준 NaN 값 대체):")
def consolidate_all_movie_info(group):
    first_row = group.iloc[0].copy()
    for col in ['multiMovieYn', 'repNationCd']:
        non_na_value = group[pd.notna(group[col])][col].iloc[0] if any(pd.notna(group[col])) else None
        if non_na_value is not None:
            first_row[col] = non_na_value
    return first_row[['movieNm', 'multiMovieYn', 'repNationCd']]

all_consolidated_df = df.groupby('movieCd').apply(consolidate_all_movie_info).reset_index()
#print(all_consolidated_df[['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']])
all_consolidated_df[['movieCd', 'movieNm', 'multiMovieYn', 'repNationCd']]

multiMovieYn, repNationCd 별 count:
multiMovieYn repNationCd  count
           N           F      0
           N           K      0
           N         NaN     10
           Y           F      0
           Y           K      0
           Y         NaN     10
         NaN           F     10
         NaN           K     10
         NaN         NaN     10
------------------------------
------------------------------
전체 영화 정보 (movieCd 기준 NaN 값 대체):


/tmp/ipykernel_2567532/1557652242.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_consolidated_df = df.groupby('movieCd').apply(consolidate_all_movie_info).reset_index()


,movieCd,movieNm,multiMovieYn,repNationCd
0,19860125,쿼바디스,Y,NaN
1,20124722,피아니스트의 전설,Y,NaN
2,20190324,3일의 휴가,N,K
3,20202247,싱글 인 서울,NaN,K
4,20203702,노량: 죽음의 바다,N,K
5,20212866,서울의 봄,N,K
6,20227525,교토에서 온 편지,NaN,K
7,20228560,너와 나,NaN,K
8,20232077,어른 김장하,NaN,K
9,20233299,크레센도,Y,F


/tmp/ipykernel_2567532/2278321826.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_consolidated_df = df.groupby('movieCd').apply(consolidate_all_movie_info).reset_index(drop=True)


KeyError: "['movieCd'] not in index"